In [1]:
import xml.etree.ElementTree as ET
from multiprocessing import Pool
import os
import requests
import json
import pafy
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import matplotlib
from matplotlib import pyplot, rcParams
%matplotlib inline
rcParams['figure.figsize'] = (20, 10)

In [2]:
# def parse_xml(tree):
#     words = []
#     for p in next(x for x in tree.getchildren() if x.tag == 'body').getchildren():
#         if p.tag == 'p':
#             offset_time = int(p.get('t'))
#             for s in p.getchildren():
#                 t = int(s.get('t', 0)) + offset_time
#                 if len(words) > 0:
#                     words[-1]['to'] = t
#                 words.append({'from': t, 'word': s.text.strip(), 'ac': s.get('ac')})
#             if len(p.getchildren()) == 0 and len(words) > 0:
#                 words[-1]['to'] = int(p.get('t'))
#     return words

def parse_xml(tree):
    for piece in tree.getchildren():
        start = float(piece.get('start'))
        yield {
            'from': start,
            'to': start + float(piece.get('dur')),
            'text': piece.text
        }

def merge_if(gen, max_time):
    grouped = [[]]
    previous = None
    for current in gen:
        if previous is not None and current['from'] - previous['to'] >= max_time:
            grouped.append([])
        grouped[-1].append(current)
        previous = current
    for group in grouped:
        yield {'from': group[0]['from'], 'to': group[-1]['to'], 'text': ' '.join(x['text'].strip() for x in group)}

def get_captions(url, language_code='nl'):
    url = url.replace('\n', '')
    response = requests.get(url)
    data = response.text
    for x in data.split('\n'):
        if x.find('timed') >= 0:
            begin = x.find('ytplayer.config = ')
            end = x.find(';ytplayer.load')
            if begin >= 0 and end >=0:
                data = json.loads(x[x.find('ytplayer.config = ') + len('ytplayer.config = '):end])
                break
    else:
        return None
    captions = json.loads(data['args']['player_response'])['captions']['playerCaptionsTracklistRenderer']['captionTracks']

    for caption in captions:
        if caption['languageCode'] == language_code:
            caption_url = caption['baseUrl'] # + '&fmt=srv3'
            caption_xml = requests.get(caption_url).text
            tree = ET.fromstring(caption_xml)
            return merge_if(parse_xml(tree), 1)

In [3]:
def split_in_pieces(url):
    id = url.split('v=')[-1].replace('\n', '')
    captions = get_captions(url)
    captions = [] if captions is None else list(captions)
    filename = 'data/rtvnoord/' + id + '.webm'
    part_filename = 'data/rtvnoord/' + id + '-part-{:04d}.webm'
    text_filename = 'data/rtvnoord/' + id + '-part-{:04d}.txt'
    if len(captions) > 0:
        for i, piece in enumerate(captions):
            ffmpeg_extract_subclip(filename, piece['from'], piece['to'], targetname=part_filename.format(i))
            with open(text_filename.format(i), 'w') as f:
                f.write(piece['text'])
            ffmpeg_extract_subclip(filename, piece['from'], piece['to'], targetname=part_filename.format(i))
        print('done', filename)
    else:
        print('no text', filename)

with open('rtvnoord', 'r') as f:
    urls = list(f)

pool = Pool(8)
pool.map(split_in_pieces, urls)

ConnectionError: None: Max retries exceeded with url: /watch?v=62fdcQM0IR0 (Caused by None)

In [4]:
def split_in_pieces(url):
    id = url.split('v=')[-1].replace('\n', '')
    captions = get_captions(url)
    captions = [] if captions is None else list(captions)
    filename = 'data/rtvoost/' + id + '.webm'
    part_filename = 'data/rtvoost/' + id + '-part-{:04d}.webm'
    if len(captions) > 0:
        for i, piece in enumerate(captions):
            ffmpeg_extract_subclip(filename, piece['from'], piece['to'], targetname=part_filename.format(i))
        print('done', filename)
    else:
        print('no text', filename)

with open('rtvoost', 'r') as f:
    urls = list(f)

pool = Pool(8)
pool.map(split_in_pieces, urls)

no text data/rtvoost/f53OkpbKWtQ.webm
no text data/rtvoost/JfBOx_a-Huk.webm
no text data/rtvoost/r_w_Iagmvi8.webm
no text data/rtvoost/DtgaqS6SJSs.webm

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/5JgGALKU9W8.webm -ss 0.00 -t 9.36 -vcodec copy -acodec copy data/rtvoost/5JgGALKU9W8-part-0000.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/m98uH7D5USs.webm -ss 0.31 -t 3.00 -vcodec copy -acodec copy data/rtvoost/m98uH7D5USs-part-0000.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/FACJbJCZWuY.webm -ss 0.92 -t 91.50 -vcodec copy -acodec copy data/rtvoost/FACJbJCZWuY-part-0000.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/tLcRf7lOox8.webm -ss 0.91 -t 69.98 -vcodec copy -acodec copy data/rtvoost/tLcRf7lOox8-part-0000.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/5JgGALKU9W8.webm -ss 12.25 -t 3.00 -vcodec copy -acodec copy data/rtvoost/5JgGALKU9W8-part-0001.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/

>>> ffmpeg -y -i data/rtvoost/xgwaYrzdw3A.webm -ss 436.38 -t 3.00 -vcodec copy -acodec copy data/rtvoost/xgwaYrzdw3A-part-0018.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/EiguG5lzUVU.webm -ss 0.40 -t 75.28 -vcodec copy -acodec copy data/rtvoost/EiguG5lzUVU-part-0000.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/0JDLlgvuDrE.webm -ss 0.66 -t 23.28 -vcodec copy -acodec copy data/rtvoost/0JDLlgvuDrE-part-0000.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/S5YVWDAfnjg.webm -ss 176.81 -t 20.78 -vcodec copy -acodec copy data/rtvoost/S5YVWDAfnjg-part-0010.webm
... command successful.

... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/nwyjOO43_q4.webm -ss 327.03 -t 7.81 -vcodec copy -acodec copy data/rtvoost/nwyjOO43_q4-part-0012.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/CBe5OFKu7JE.webm -ss 179.85 -t 4.17 -vcodec copy -acodec copy data/rtvoost/CBe5OFKu7JE-part-0001.webm
... command successful.

... command su

no text data/rtvoost/sn6KkLAgFhY.webm

... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/qaUlT4jrNmc.webm -ss 488.24 -t 12.29 -vcodec copy -acodec copy data/rtvoost/qaUlT4jrNmc-part-0005.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/EG1VBKPxq0I.webm -ss 0.00 -t 163.22 -vcodec copy -acodec copy data/rtvoost/EG1VBKPxq0I-part-0000.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/xgwaYrzdw3A.webm -ss 681.44 -t 6.12 -vcodec copy -acodec copy data/rtvoost/xgwaYrzdw3A-part-0036.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/qaUlT4jrNmc.webm -ss 501.81 -t 176.90 -vcodec copy -acodec copy data/rtvoost/qaUlT4jrNmc-part-0006.webmno text data/rtvoost/JvWRlwJuG4w.webm

... command successful.
done data/rtvoost/LMHgEsh81to.webm

... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/xgwaYrzdw3A.webm -ss 690.92 -t 18.55 -vcodec copy -acodec copy data/rtvoost/xgwaYrzdw3A-part-0037.webm
... 

>>> ffmpeg -y -i data/rtvoost/xgwaYrzdw3A.webm -ss 863.16 -t 3.00 -vcodec copy -acodec copy data/rtvoost/xgwaYrzdw3A-part-0050.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/6YxbvdHDHkk.webm -ss 20.86 -t 8.49 -vcodec copy -acodec copy data/rtvoost/6YxbvdHDHkk-part-0001.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/KyahWX5UZdY.webm -ss 222.36 -t 3.00 -vcodec copy -acodec copy data/rtvoost/KyahWX5UZdY-part-0009.webm
... command successful.

... command successful.
done data/rtvoost/b7jxQuq_kKs.webm

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/GbEQvnZ86zc.webm -ss 28.33 -t 3.00 -vcodec copy -acodec copy data/rtvoost/GbEQvnZ86zc-part-0002.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/RygIwu-E_Xw.webm -ss 87.45 -t 15.03 -vcodec copy -acodec copy data/rtvoost/RygIwu-E_Xw-part-0003.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/6YxbvdHDHkk.webm -ss 30.50 -t 44.61 -vcodec copy -acodec 

>>> ffmpeg -y -i data/rtvoost/xgwaYrzdw3A.webm -ss 1135.82 -t 9.56 -vcodec copy -acodec copy data/rtvoost/xgwaYrzdw3A-part-0067.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/wDUyx06vE6k.webm -ss 25.58 -t 6.54 -vcodec copy -acodec copy data/rtvoost/wDUyx06vE6k-part-0001.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/SeMwcfRZmWA.webm -ss 219.47 -t 14.56 -vcodec copy -acodec copy data/rtvoost/SeMwcfRZmWA-part-0008.webm
[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/f_O25K1XOnQ.webm -ss 0.39 -t 22.04 -vcodec copy -acodec copy data/rtvoost/f_O25K1XOnQ-part-0000.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/Z4bM0N6oTa8.webm -ss 112.54 -t 20.82 -vcodec copy -acodec copy data/rtvoost/Z4bM0N6oTa8-part-0003.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/wDUyx06vE6k.webm -ss 33.63 -t 3.69 -vcodec copy -acodec copy data/rtvoost/wDUyx06vE6k-part-0002.webm
... command s

...

>>> ffmpeg -y -i data/rtvoost/6MO8ofslrpM.webm -ss 176.37 -t 86.10 -vcodec copy -acodec copy data/rtvoost/6MO8ofslrpM-part-0001.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/6MO8ofslrpM.webm -ss 263.80 -t 17.05 -vcodec copy -acodec copy data/rtvoost/6MO8ofslrpM-part-0002.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/6MO8ofslrpM.webm -ss 282.73 -t 33.76 -vcodec copy -acodec copy data/rtvoost/6MO8ofslrpM-part-0003.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/6MO8ofslrpM.webm -ss 318.03 -t 31.23 -vcodec copy -acodec copy data/rtvoost/6MO8ofslrpM-part-0004.webm
... command successful.
done data/rtvoost/6MO8ofslrpM.webm
no text data/rtvoost/SMm5rAFioQk.webm

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/qeqc9xevBiI.webm -ss 8.46 -t 65.56 -vcodec copy -acodec copy data/rtvoost/qeqc9xevBiI-part-0000.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/qeqc9xevBiI.we

>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 31.26 -t 110.51 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0001.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 142.97 -t 33.51 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0002.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 177.59 -t 7.52 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0003.webmno text data/rtvoost/DOkeHyA9Qtk.webm

... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 186.12 -t 24.26 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0004.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 211.82 -t 10.46 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0005.webmno text data/rtvoost/vJ7ayvhfOmI.webm

... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2q


[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 1071.42 -t 30.18 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0036.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/25Q2ApnB2qs.webm -ss 1102.87 -t 3.26 -vcodec copy -acodec copy data/rtvoost/25Q2ApnB2qs-part-0037.webm
... command successful.
done data/rtvoost/25Q2ApnB2qs.webm
no text data/rtvoost/obwj2Ap8MlQ.webm

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/FLyZhj0KOag.webm -ss 3.35 -t 126.84 -vcodec copy -acodec copy data/rtvoost/FLyZhj0KOag-part-0000.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/FLyZhj0KOag.webm -ss 131.28 -t 140.90 -vcodec copy -acodec copy data/rtvoost/FLyZhj0KOag-part-0001.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/FLyZhj0KOag.webm -ss 273.29 -t 25.75 -vcodec copy -acodec copy data/rtvoost/FLyZhj0KOag-part-0002.webm
... command successful.
done data/rtvoost/FLyZhj0KOag.webm
no tex

>>> ffmpeg -y -i data/rtvoost/gL3MDgHwsqk.webm -ss 633.74 -t 67.30 -vcodec copy -acodec copy data/rtvoost/gL3MDgHwsqk-part-0011.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/gL3MDgHwsqk.webm -ss 702.22 -t 140.80 -vcodec copy -acodec copy data/rtvoost/gL3MDgHwsqk-part-0012.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/gL3MDgHwsqk.webm -ss 844.97 -t 40.82 -vcodec copy -acodec copy data/rtvoost/gL3MDgHwsqk-part-0013.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/gL3MDgHwsqk.webm -ss 888.13 -t 8.16 -vcodec copy -acodec copy data/rtvoost/gL3MDgHwsqk-part-0014.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/gL3MDgHwsqk.webm -ss 898.88 -t 11.07 -vcodec copy -acodec copy data/rtvoost/gL3MDgHwsqk-part-0015.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/gL3MDgHwsqk.webm -ss 922.48 -t 4.01 -vcodec copy -acodec copy data/rtvoost/gL3MDgHwsq

>>> ffmpeg -y -i data/rtvoost/jpZcGSMfAMU.webm -ss 1118.52 -t 3.00 -vcodec copy -acodec copy data/rtvoost/jpZcGSMfAMU-part-0026.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/jpZcGSMfAMU.webm -ss 1123.66 -t 20.32 -vcodec copy -acodec copy data/rtvoost/jpZcGSMfAMU-part-0027.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/jpZcGSMfAMU.webm -ss 1145.05 -t 6.02 -vcodec copy -acodec copy data/rtvoost/jpZcGSMfAMU-part-0028.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/jpZcGSMfAMU.webm -ss 1155.06 -t 5.31 -vcodec copy -acodec copy data/rtvoost/jpZcGSMfAMU-part-0029.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/jpZcGSMfAMU.webm -ss 1164.65 -t 14.23 -vcodec copy -acodec copy data/rtvoost/jpZcGSMfAMU-part-0030.webm
... command successful.

[MoviePy] Running:
>>> ffmpeg -y -i data/rtvoost/jpZcGSMfAMU.webm -ss 1180.01 -t 48.80 -vcodec copy -acodec copy data/rtvoost/jpZcGS